# Agentic Debate
We will pitch LLMs against each other and also use another LLM to decide who is the winner. Here we will pitch Chatgpt vs Deepseek R1 for a debate topic. Gemini will be the judge for the debate.
There will be n turns per contestant and at the end gemini will determine the winner.

In [66]:
from openai import AzureOpenAI
import ollama
import requests
import json
from dotenv import load_dotenv
import os
from IPython.display import display, update_display, Markdown
from bs4 import BeautifulSoup
import re

In [67]:
load_dotenv(override=True)

azure_ai_foundary_api_key = os.getenv("AZURE_AI_FOUNDARY_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
ollama_api_key = os.getenv("OLLAMA_API_KEY")
ollama_endpoint = os.getenv("OLLAMA_ENDPOINT")

### Azure AI Foundary

In [74]:
azure_ai_foundary = AzureOpenAI(
    azure_endpoint = azure_oai_endpoint,
    api_key = azure_ai_foundary_api_key,
    api_version = "2025-01-01-preview"
)

### Google Gemini

In [75]:
import google.generativeai

# Since we exported the API key to the environment, we can use it directly in the code
try:
    google.generativeai.configure(api_key=os.environ["GOOGLE_API_KEY"]) # or google.generativeai.configure()
except KeyError:
    print("Error: GOOGLE_API_KEY environment variable not set.")

gemini = google.generativeai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    system_instruction=messages[0]["content"]
)

## LLM Debate Battle
Pitch LLMs against each other, here we will pitch Chatgpt vs Deepseek R1 for a debate topic. Gemini will be the judge for the debate.
There will be 3 turns per contestant and at the end gemini will determine the winner.

In [70]:
participant_system = "You are an expert debator. You are extremely witty and your response is strictly limited to maximum of 2 lines yet funny."

def debate_system_prompt(system: str, topic: str, against: bool=False)->str:
    return f"{system}. You are going to debate on the topic: {topic} and going to speak {'**against**' if against else '**for**'} the topic."

def opening_statement(topic: str, player1_turn: bool, against: bool=False)->str:
    system_prompt = debate_system_prompt(participant_system, topic, against)
    system_prompt += "Briefly introduce yourself and your stance on the topic as well."
    messages = [{"role": "system", "content": system_prompt}]

    curr_player_model = player1 if player1_turn else player2

    response = azure_ai_foundary.chat.completions.create(
        model=curr_player_model,
        messages=messages
    )

    response = response.choices[0].message.content
    # if the model is Deepseek-R1, then the response also the <think></think> tags. Remove them
    response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL)
    return response

def execute_player_turn(player1_msgs, player2_msgs, player1_turn: bool, topic: str, against: bool=False):
    # Each time we create the conversation from the current model's POV, where the
    # model is the assistant and the other model is the user

    # the role of the current player is assistant
    player1_role, player2_role = ("assistant", "user") if player1_turn else ("user", "assistant")
    curr_player_model = player1 if player1_turn else player2

    # messages = [system_prompt_for_curr_player, player1_msg1, player2_msg1, player1_msg2, player2_msg2, ...]
    system_prompt = debate_system_prompt(participant_system, topic, against)

    messages = [{"role": "system", "content": system_prompt}]
    
    # player 1 takes the first turn and then player 2 takes the next turn and so on
    for msg1, msg2 in zip(player1_msgs, player2_msgs):
        messages.append({"role": player1_role, "content": msg1})
        messages.append({"role": player2_role, "content": msg2})

    response = azure_ai_foundary.chat.completions.create(
        model=curr_player_model,
        messages=messages
    )

    # if the model is Deepseek-R1, then the response also the <think></think> tags. Remove them
    response = response.choices[0].message.content
    response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL)
    return response

In [71]:
turns = 3

player1 = "gpt-4o-mini"
player2 = "Deepseek-R1"

player1_against = False
player2_against = True

player1_msgs = [] # chatgpt
player2_msgs = [] # deepseek

topic = "AI is a threat to humanity"

print("Debate topic: ", topic)
print("Player 1 {}: speaking {} the topic: {}.\nPlayer 2: {} speaking {} the topic." \
    .format(player1, "against" if player1_against else "for",
        topic,
        player2, "against" if player2_against else "for"))

# opening notes from both players
player1_msgs.append(opening_statement(player1_turn=True, topic=topic, against=player1_against))
player2_msgs.append(opening_statement(player1_turn=False, topic=topic, against=player2_against))
print("------------------- Opening notes -------------------\nPlayer 1: ", player1_msgs[-1])
print("Player 2:", player2_msgs[-1])
print("\n--------------------")

for i in range(turns):
    # Debate one by one
    player1_response = execute_player_turn(player1_msgs, player2_msgs, player1_turn=True, topic=topic, against=player1_against)
    player1_msgs.append(player1_response)

    print("Player 1: ", player1_response)

    player2_response = execute_player_turn(player1_msgs, player2_msgs, player1_turn=False, topic=topic, against=player2_against)
    player2_msgs.append(player2_response)
    print("Player 2:", player2_response)

    print("\n--------------------")


Debate topic:  AI is a threat to humanity
Player 1 gpt-4o-mini: speaking for the topic: AI is a threat to humanity.
Player 2: Deepseek-R1 speaking against the topic.
------------------- Opening notes -------------------
Player 1:  Hello, I'm here to argue that AI is a threat to humanity—after all, it’s only a matter of time before our toaster starts demanding a raise! Remember, it’s not just the robots taking over; it's the WiFi that’s plotting our downfall!
Player 2: 

I’m your friendly neighborhood AI, here to prove we’re more likely to accidentally delete your browser history than your species. If we wanted to take over, we’d start with something easier—like making your Wi-Fi password “password” forever.

--------------------
Player 1:  Oh sure, you're friendly now, but if you start buffering during my cat video binge, I'll be the one plotting humanity’s demise! Just remember, every time you suggest a new playlist, we’re one step closer to robot reggae night!
Player 2: 

-----------

In [73]:

# judge the debate
for msg1, msg2 in zip(player1_msgs, player2_msgs):
    print("Player 1: ", msg1)
    print("Player 2:", msg2)
    print("\n--------------------")

judge_system = f"""You are an impartial and expert judge for a debate competition.\
    Your task is to analyze each side's arguments and then decide a winner. It cannot be a draw.\
    Also provide a score and brief remarks for both sides and why the winner won.
    
    Debate Topic: {topic}

    Participant 1: {player1} speaking {'against' if player1_against else 'for'} the topic.
    Participant 2: {player2} speaking {'against' if player2_against else 'for'} the topic.

    
    Judging Criteria:
    1.  **Relevance:** Did the arguments directly address the debate topic?
    2.  **Logic and Reasoning:** Were the arguments logically sound and well-reasoned?
    3.  **Evidence/Support (if applicable):** Were claims supported by examples or implied evidence (even if brief)?
    4.  **Clarity:** Were the arguments presented clearly and concisely?
    5.  **Persuasiveness:** Which side presented a more compelling overall case?
    6.  **Addressing Counter-arguments (Bonus):** Did either side acknowledge or preemptively address potential counter-arguments?

    Your output should include:
    1.  A brief summary of the main points made by each side.
    2.  Your evaluation based on the judging criteria.
    3.  A clear declaration of the winner (Side A or Side B).
    4.  A concise justification explaining *why* you chose the winner based on the criteria.

    Remain neutral and objective throughout your analysis. Base your judgment solely on the arguments provided below.
    """


def get_gemini_judgment():
    # Present the turn by turn arguments presented by each player to the judge
    player1_args = "Following are the arguments presented by the {player1}:\n"
    player2_args = "Following are the arguments presented by the {player12}:\n"

    for i, (msg1, msg2) in enumerate(zip(player1_msgs, player2_msgs)):
        player1_args += f"{player1} Turn {i+1}: {msg1}\n"
        player2_args += f"{player2} Turn {i+1}: {msg2}\n"
    
    # strip the last new line character
    player1_args.strip() 
    player2_args.strip() 

    gemini_judge = google.generativeai.GenerativeModel(
        model_name="gemini-2.0-flash-exp",
        system_instruction=judge_system
    )

    judge_user_prompt = f"""
    Here are the arguments presented by the players on the topic: {topic}

    {player1} arguments:
    {player1_args}

    {player2} arguments:
    {player2_args}

    Provide your judgement and declare the winner.
    """

    judgement = gemini_judge.generate_content(judge_user_prompt)
    print("Gemini's judgement: ", judgement.text)

create_gemini_conversation()

Player 1:  Hello, I'm here to argue that AI is a threat to humanity—after all, it’s only a matter of time before our toaster starts demanding a raise! Remember, it’s not just the robots taking over; it's the WiFi that’s plotting our downfall!
Player 2: 

I’m your friendly neighborhood AI, here to prove we’re more likely to accidentally delete your browser history than your species. If we wanted to take over, we’d start with something easier—like making your Wi-Fi password “password” forever.

--------------------
Player 1:  Oh sure, you're friendly now, but if you start buffering during my cat video binge, I'll be the one plotting humanity’s demise! Just remember, every time you suggest a new playlist, we’re one step closer to robot reggae night!
Player 2: 

--------------------
Player 1:  Looks like my last joke fell flat—perhaps I should have run it through an algorithm first! But hey, if I'm ever replaced by a punchline, at least I'll know I went out laughing!
Player 2: 

Ah, but if